In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CADMium import Pssolver
from CADMium import Psgrid
from CADMium import Partition
from CADMium import Inverter

a = 5.122/2
#Nuclear charge for fragments A and B
Za, Zb = 3,3
#Set polarization 1-Unpolarized, 2-Polarized
pol = 2
#Fragment a electrons [alpha, beta]
Nmo_a = [[2,1]] #Number of molecular orbitals to calculate
N_a   = [[2,1]]
#Ensemble mix
nu_a = 1
#Fragment b electrons
Nmo_b = [[2,1]]
N_b   = [[2,1]]
#Ensemble mix
nu_b = 1

#Molecular elctron configuration
Nmo_m = [[3,3]]
N_m   = [[3,3]]
 

#Set up grid
NP = 7
NM = [4,4]
L = np.arccosh(15/a)
loc = np.array(range(-4,5)) #Stencil outline

grid = Psgrid(NP, NM, a, L, loc)
grid.initialize()

part = Partition(grid, Za, Zb, pol, Nmo_a, N_a, nu_a, Nmo_b, N_b, nu_b, { "AB_SYM"       : True,
                                                                          "ENS_SPIN_SYM" : True,})

#Setup inverter object
mol_solver = Pssolver(grid, Nmo_m, N_m)
part.inverter = Inverter(grid, mol_solver, {"invert_type"  : "orbitalinvert",
                                            "disp"         : True,
                                            "AB_SYM"       : True,
                                            "ENS_SPIN_SYM" : True,})

part.optPartition.isolated = True
part.scf({"disp" : True,
          "e_tol" : 1e-5})

 Constructing integration weights ... 

 Building finite difference opperators ... 

 Factorizing Laplacian ... 

 Done


In [2]:
part.KSa.solver[0][0].optSolver.iter_lin_solver = False
part.KSa.solver[0][1].optSolver.iter_lin_solver = False

In [3]:
part.optPartition.isolated   = False

part.scf({"disp"       : True,
          "e_tol"      : 1e-8,
          "continuing" : True})



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
one = np.array([1,2,3,4,5])[:,None]
two = np.repeat(one, 2, axis=1)
two / np.repeat(one, two.shape[1], axis=1)

In [ ]:
np.repeat(one, two.shape[1], axis=1)

In [ ]:
full, x,y = grid.plotter(part.V.vp[:,0] + part.V.vp[:,1])
x_axis,vp = grid.axis_plot(part.V.vp[:,0] + part.V.vp[:,1])

In [ ]:
fig = plt.figure(dpi=150)
plt.contourf(x,y,full, levels=2, cmap="viridis")
# plt.xlim([-5,5])
# plt.ylim([-5,5])
plt.colorbar()

In [ ]:
fig = plt.figure(dpi=120)
plt.plot(x_axis,vp)
plt.xlim(0,5)